In [10]:
DATA_DIR = "/content/drive/MyDrive/DocumentUpskilling/dataset/documents"

## Import

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import cv2

In [12]:
model_name = "squeezenet"

num_classes = 4

batch_size = 2

num_epochs = 5

feature_extract = True

# Training Model

In [13]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'test':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [14]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [15]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/squeezenet1_0-b66bff10.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_0-b66bff10.pth
100%|██████████| 4.78M/4.78M [00:00<00:00, 38.6MB/s]

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [17]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 classifier.1.weight
	 classifier.1.bias


In [18]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(DATA_DIR, x), data_transforms[x]) for x in ['train', 'test']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'test']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [19]:
# Assuming image_datasets is already defined as in your code
for phase, dataset in image_datasets.items():
    print(f"Class names in {phase} set: {dataset.classes}")


Class names in train set: ['citizenship', 'license', 'others', 'passport']
Class names in test set: ['citizenship', 'license', 'others', 'passport']


In [20]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=10, is_inception=(model_name=="inception"))

Epoch 0/9
----------
train Loss: 1.5644 Acc: 0.2931
test Loss: 1.0737 Acc: 0.5000

Epoch 1/9
----------
train Loss: 1.0216 Acc: 0.5862
test Loss: 0.9188 Acc: 0.6250

Epoch 2/9
----------
train Loss: 0.9517 Acc: 0.6379
test Loss: 0.6567 Acc: 0.7500

Epoch 3/9
----------
train Loss: 0.8461 Acc: 0.6897
test Loss: 0.5956 Acc: 0.7500

Epoch 4/9
----------
train Loss: 0.6340 Acc: 0.7931
test Loss: 0.4887 Acc: 0.8750

Epoch 5/9
----------
train Loss: 0.5104 Acc: 0.7931
test Loss: 0.3434 Acc: 0.8750

Epoch 6/9
----------
train Loss: 0.4709 Acc: 0.7931
test Loss: 0.3106 Acc: 0.8750

Epoch 7/9
----------
train Loss: 0.4361 Acc: 0.8793
test Loss: 0.2105 Acc: 0.9375

Epoch 8/9
----------
train Loss: 0.6516 Acc: 0.7586
test Loss: 0.3069 Acc: 0.8750

Epoch 9/9
----------
train Loss: 0.3486 Acc: 0.8621
test Loss: 0.2595 Acc: 0.9375

Training complete in 1m 21s
Best val Acc: 0.937500


# Export as onnx model

In [21]:
!pip install onnx
!pip install onnxscript

In [22]:
torch_model = model_ft
torch_input = torch.randn(1, 3, 224, 224)
onnx_program = torch.onnx.dynamo_export(torch_model, torch_input)

/usr/local/lib/python3.10/dist-packages/torch/onnx/_internal/exporter.py:130: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


In [23]:
onnx_program.save("doc_classifier.onnx")

In [24]:
import onnx
onnx_model = onnx.load("doc_classifier.onnx")
onnx.checker.check_model(onnx_model)

# Inference using onnx model

In [4]:
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.7 MB/s eta 0:00:00


In [5]:
import onnxruntime
import numpy as np

In [32]:
def predict_using_onnx(input_img):

  onnx_input = onnx_program.adapt_torch_inputs_to_onnx(input_img)
  ort_session = onnxruntime.InferenceSession("doc_classifier.onnx", providers=['CPUExecutionProvider'])

  def to_numpy(tensor):
      return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

  onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)}

  onnxruntime_outputs = ort_session.run(None, onnxruntime_input)

  max_index = np.argmax(onnxruntime_outputs)

  class_names =  ['citizenship', 'license', 'others', 'passport']

  predicted_class = class_names[max_index]

  return predicted_class

In [35]:
import torch
from torchvision import transforms
from PIL import Image

def predict_image(image_path, actual_class):
    # Load the image
    img = Image.open(image_path).convert("RGB")


    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Apply transformations
    img = transform(img)



    # Add batch dimension to the image
    img = img.unsqueeze(0)

    # Move the image to the appropriate device (CPU or GPU)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    img = img.to(device)

    onnx_output = predict_using_onnx(img)

    return onnx_output, img

# Example usage
image_path = "/content/drive/MyDrive/DocumentUpskilling/dataset/blurred_citizenship/test/blur/CpvQJpwWIAAGJEd.jpg"
predicted_class,img = predict_image(image_path, actual_class="citizenship")

In [29]:
print(predicted_class)

citizenship


In [8]:
%cd /content/drive/MyDrive/DocumentUpskilling/test_scripts

/content/drive/MyDrive/DocumentUpskilling/test_scripts


In [18]:
%%writefile doc_classify.py
"""
Script for classifying images using an ONNX model.

Usage:
python doc_classify.py <model_path> <image_path>

Arguments:
- model_path (str): Path to the ONNX model file.
- image_path (str): Path to the image to be classified.

Example:
python doc_classify.py model.onnx image.jpg
"""

import json
import torch
from torchvision import transforms
from PIL import Image
import onnxruntime
import numpy as np

def transform_image(image_path):
    """
    Preprocesses the input image for classification.

    Args:
        image_path (str): Path to the input image.

    Returns:
        torch.Tensor: Preprocessed image tensor.
    """
    img = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Apply transformations
    img = transform(img)

    # Add batch dimension to the image
    img = img.unsqueeze(0)

    # Move the image to the appropriate device (CPU or GPU)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    img = img.to(device)
    return img

def classify_image(model_path, image_path):
    """
    Classifies the input image using the specified ONNX model.

    Args:
        model_path (str): Path to the ONNX model file.
        image_path (str): Path to the image to be classified.

    Returns:
        str: Predicted class label.
    """
    # load model
    session = onnxruntime.InferenceSession(model_path, None)

    # get input name from the model
    input_name = session.get_inputs()[0].name

    # get output name from the model
    output_name = session.get_outputs()[0].name

    # transform image for the model
    img = transform_image(image_path)

    # convert image into list
    data = json.dumps({'data': img.tolist()})
    data = np.array(json.loads(data)['data']).astype('float32')

    # feed the image into the model
    result = session.run([output_name], {input_name: data})
    max_index = np.argmax(result)
    class_names = ['citizenship', 'license', 'others', 'passport']

    predicted_class = class_names[max_index]
    return predicted_class

Writing doc_classify.py


In [13]:
%cd /content/drive/MyDrive/DocumentUpskilling

/content/drive/MyDrive/DocumentUpskilling


In [14]:
from test_scripts.doc_classify import classify_image

In [15]:
res = classify_image("/content/doc_classifier.onnx", "/content/drive/MyDrive/DocumentUpskilling/dataset/blurred_citizenship/test/blur/CpvQJpwWIAAGJEd.jpg")

In [17]:
res

'citizenship'